# Importing Libraries

In [64]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service

# Open the chrome and call the Website

In [65]:
driver = webdriver.Chrome(executable_path= r"C:\api\chromedriver.exe")
driver.get('https://portal.wbsec.org/')
driver.maximize_window()


C:\Users\Deepak Verma\AppData\Local\Temp\ipykernel_9956\983227409.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= r"C:\api\chromedriver.exe")


# finding the number of constituency


In [66]:
city_list = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div[1]/div/section/div/div/div/div/div[2]/div[2]/select").text.split("\n")[:-1]
final_city_list = [i.strip() for i in city_list]
final_city_list

['COOCHBEHAR',
 'JALPAIGURI',
 'UTTAR DINAJPUR',
 'DAKSHIN DINAJPUR',
 'MALDA',
 'MURSHIDABAD',
 'NADIA',
 'NORTH 24 PARGANAS',
 'SOUTH 24-PARGANAS',
 'HOWRAH',
 'HOOGHLY',
 'PURBA MEDINIPUR',
 'PASCHIM MEDINIPUR',
 'PURULIA',
 'BANKURA',
 'PURBA BARDHAMAN',
 'BIRBHUM',
 'ALIPURDUAR',
 'JHARGRAM',
 'PASCHIM BARDHAMAN',
 'KALIMPONG',
 'DARJEELING']

# Candidate Info


In [67]:
''' Defining a function which will extract all the info of candidate from a single page this function will take soup of 
the table and class tag and name of the city'''
def candidate_info(table_soup, class_tr, district):
    # creating one dummy list which will store all the data 
    odd_list = []
    # finding all the data of tr tag
    odd_tr = table_soup.find("tbody", class_ = "ng-scope").find_all("tr", class_ = class_tr)
    # running a loop for tr tag
    for i in range(len(odd_tr)):
        sno = odd_tr[i].find_all("td")[0].text # S.no finding 
        name = odd_tr[i].find_all("td")[1].find('span').text # Name finding
        gen = odd_tr[i].find_all("td")[2].text.strip() # Gender finding
        add = odd_tr[i].find_all("td")[3].text.strip() # Address finding
        party = odd_tr[i].find_all("td")[4].text.strip() # Party finding
        cat = odd_tr[i].find_all("td")[5].text.strip() # Category finding
        # creating one dictonary which will store all the info
        dic_info = {"district":district, "sno": sno, "name": name, "gen": gen, "add": add, "party": party, "cat": cat}     
        odd_list.append(dic_info) # appending the dictonary into emptry list
    return odd_list # returning the list

In [68]:
def search_firstpage_district(cat):
    data = []
    # selecting the Constituency
    #{"Zila Parishad": 0, "Panchayat samity": 1, "gram Panchayat": 2}
    Select(driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div[1]/div/section/div/div/div/div/div[2]/div[1]/select")).select_by_index(0)
    # selecting the category (It will change by the index args)
    Select(driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div[1]/div/section/div/div/div/div/div[2]/div[2]/select")).select_by_visible_text(cat)
    # click on the search button
    driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div[1]/div/section/div/div/div/div/div[2]/div[5]/button").click()
    sleep(3) # change accordingly
    # setting number of results to 100
    Select(driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div[1]/div/section/div/div/div/div/div[5]/div/div/div/div[1]/div[1]/div/label/select")).select_by_value("100")
    # get the all page source
    soup = BeautifulSoup(driver.page_source)
    # finding the table soup
    table_soup = soup.find("table", id = "tblContestingCandidatesAcceptedPortalList")
    # providing the tag list
    tag_list = ["ng-scope even", "ng-scope odd"]
    for i in tag_list:
        data += candidate_info(table_soup, i, cat)
    #finding total_entries
    total_entries = soup.find("div", class_ = "dataTables_info").text.split()[-2]
    print(total_entries)
    # finding the number of page
    number_page = int(total_entries)//100+1
    while number_page > 1:
        driver.find_element(By.ID, "tblContestingCandidatesAcceptedPortalList_next").click()
        sleep(2)
        soup = BeautifulSoup(driver.page_source)
        # finding the table soup
        table_soup = soup.find("table", id = "tblContestingCandidatesAcceptedPortalList")
        # providing the tag list
        tag_list = ["ng-scope even", "ng-scope odd"]
        for i in tag_list:
            data += candidate_info(table_soup, i, cat)
        number_page -= 1
    return data

In [70]:
final_data = [] # dummy final_data list
not_found_list = [] # dummy not_found_list list
for i in final_city_list: # running for loop for whole cities
    try:
        data= search_firstpage_district(i)
        final_data += data
    except:
        data = [0]
        not_found_list.append(i)
    sleep(1)
    print(i,len(data),  "completed")

211
COOCHBEHAR 211 completed
123
JALPAIGURI 123 completed
145
UTTAR DINAJPUR 145 completed
93
DAKSHIN DINAJPUR 93 completed
276
MALDA 276 completed
438
MURSHIDABAD 438 completed
270
NADIA 270 completed
278
NORTH 24 PARGANAS 278 completed
356
SOUTH 24-PARGANAS 356 completed
177
HOWRAH 177 completed
223
HOOGHLY 223 completed
372
PURBA MEDINIPUR 372 completed
281
PASCHIM MEDINIPUR 281 completed
279
PURULIA 279 completed
254
BANKURA 254 completed
262
PURBA BARDHAMAN 262 completed
182
BIRBHUM 182 completed
82
ALIPURDUAR 82 completed
99
JHARGRAM 99 completed
67
PASCHIM BARDHAMAN 67 completed
KALIMPONG 1 completed
DARJEELING 1 completed


In [71]:
df = pd.DataFrame(final_data)

In [72]:
df

,district,sno,name,gen,add,party,cat
0,COOCHBEHAR,2,MINA ROY,Female,"Vill:147 Bhotbari, PO: 147 Bhotbari, P.S: Mekl...",OTHERS,SC
1,COOCHBEHAR,4,JHARNA RAY,Female,"Vill: 153 Panisala, PO: Doradabri, PS: Mekliga...",CPI(M),SC
2,COOCHBEHAR,6,BHARATI ROY,Female,"Vill.- 147 Bhotbari, PO.- Bhotbari, PS.-Mekhl...",INC,SC
3,COOCHBEHAR,2,GOPAL CHANDRA ROY,Male,"Vill.- 165 Uchhalpukuri , PO.- Uchhalpukuri , ...",IND,SC
4,COOCHBEHAR,4,JAYA RISHI,Female,"Vill.-208 Buraburi Dwarikamari, PO.- Jamaldah...",IND,SC
...,...,...,...,...,...,...,...
4463,PASCHIM BARDHAMAN,1,CHAINA MALAKAR,Female,"Netaji Colony, P.O.-Shiakulberia, Jitpur Uttar...",CPI(M),SC
4464,PASCHIM BARDHAMAN,3,PAYEL BOURI,Female,"Dharmapara, Ethora, Salanpur, District-Paschim...",BJP,SC
4465,PASCHIM BARDHAMAN,2,MALA BOURI,Female,"Kadam Dhawra, Panchgachia, Kanyapur, Asansol (...",AITC,SC
4466,PASCHIM BARDHAMAN,1,BAISI MADDI,Female,"Vill.-Bhanadhaura, P.O.-Charanpur, Barabani, D...",CPI(M),ST


In [73]:
df.to_csv("data.csv", index= False)